# Cvičení 01 — Perceptron, lineární separabilita a XOR

**Cíle cvičení**
- Naučit se základy práce v Jupyteru (spouštění buněk, edit/command mód, klávesové zkratky).
- Naimplementovat **jednoduchý perceptron** v `numpy` a **natrénovat jej** na datech AND/OR.
- **Ukázat selhání na XOR** a vizualizovat rozhodovací hranici.
- Bonus (pokud zbyde čas): použít `sklearn.neural_network.MLPClassifier` a **vyřešit XOR** dvouvrstvou sítí.

> Tip: Běhejte buňky postupně shora dolů: `Shift+Enter`. Přepnutí mezi edit/command módem: `Enter` / `Esc`.

## 0) Jupyter rychlostart
- Běh buňky: **Shift+Enter**
- Vložit novou buňku pod: **B** *(command mód)*
- Smazat buňku: **DD** *(dvakrát D v command módu)*
- Přepnout buňku na Markdown: **M** / na kód: **Y**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# užitečné nastavení tisku
np.set_printoptions(suppress=True, linewidth=120)

## 1) Mini-datové sady: AND, OR, XOR

In [ ]:
# Čtyři body v {0,1}^2
X = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]], dtype=float)

y_and = np.array([0,0,0,1], dtype=float)
y_or  = np.array([0,1,1,1], dtype=float)
y_xor = np.array([0,1,1,0], dtype=float)

X

## 2) Jednoduchý perceptron (implementace v `numpy`)

In [ ]:
class Perceptron:
    def __init__(self, lr=0.1, n_epochs=50, add_bias=True, random_state=0):
        self.lr = lr
        self.n_epochs = n_epochs
        self.add_bias = add_bias
        self.rng = np.random.default_rng(random_state)
        self.w = None  # bude nastaveno při fit

    def _add_bias(self, X):
        if not self.add_bias:
            return X
        ones = np.ones((X.shape[0], 1))
        return np.hstack([ones, X])

    def predict_raw(self, X):
        Xb = self._add_bias(X)
        return Xb @ self.w

    def predict(self, X):
        # výstup 0/1 podle prahu 0
        return (self.predict_raw(X) >= 0).astype(float)

    def fit(self, X, y, verbose=False):
        Xb = self._add_bias(X)
        n_features = Xb.shape[1]
        # malé náhodné váhy
        self.w = self.rng.normal(scale=0.01, size=n_features)

        history = []
        for epoch in range(self.n_epochs):
            errors = 0
            # online učení (cyklujeme přes vzorky)
            for xi, target in zip(X, y):
                xb = self._add_bias(xi.reshape(1, -1))
                y_hat = (xb @ self.w >= 0).astype(float)[0]
                update = self.lr * (target - y_hat)
                if update != 0:
                    self.w += update * xb.ravel()
                    errors += 1
            # uložme si počet chyb za epochu
            history.append(errors)
            if verbose:
                print(f"epoch {epoch+1:02d} | errors = {errors}")
            # pokud už žádné chyby, můžeme skončit
            if errors == 0:
                break
        return history

## 3) Trénink na AND a OR

In [ ]:
pp_and = Perceptron(lr=0.2, n_epochs=50, random_state=1)
hist_and = pp_and.fit(X, y_and, verbose=True)
print("váhy AND:", pp_and.w)
print("predikce AND:", pp_and.predict(X))

In [ ]:
pp_or = Perceptron(lr=0.2, n_epochs=50, random_state=2)
hist_or = pp_or.fit(X, y_or, verbose=True)
print("váhy OR:", pp_or.w)
print("predikce OR:", pp_or.predict(X))

### Vizualizace rozhodovací hranice (2D)

In [ ]:
def plot_decision_boundary(model, X, y, title="Decision boundary"):
    # mřížka
    x_min, x_max = X[:,0].min() - 0.5, X[:,0].max() + 0.5
    y_min, y_max = X[:,1].min() - 0.5, X[:,1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    grid = np.c_[xx.ravel(), yy.ravel()]
    zz = model.predict(grid).reshape(xx.shape)

    plt.figure()
    plt.contourf(xx, yy, zz, alpha=0.3, levels=[-0.1, 0.5, 1.1])
    # trénovací body
    plt.scatter(X[:,0], X[:,1], c=y, edgecolors="k")
    plt.title(title)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.show()

plot_decision_boundary(pp_and, X, y_and, title="Perceptron — AND")
plot_decision_boundary(pp_or,  X, y_or,  title="Perceptron — OR")

## 4) XOR: ukázka selhání perceptronu

In [ ]:
pp_xor = Perceptron(lr=0.2, n_epochs=200, random_state=42)
hist_xor = pp_xor.fit(X, y_xor, verbose=True)
print("počet chyb za epochy (XOR):", hist_xor)
print("predikce XOR:", pp_xor.predict(X))

plot_decision_boundary(pp_xor, X, y_xor, title="Perceptron — XOR (selže)")

## 5) Bonus: dvouvrstvá síť (MLP) vyřeší XOR

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(4,),max_iter=1000,alpha=1e-6,random_state=0)

mlp.fit(X, y_xor.astype(int))
print("predikce MLP na XOR:", mlp.predict(X))

class Wrapper:
    def predict(self, X):
        return mlp.predict(X).astype(float)

plot_decision_boundary(Wrapper(), X, y_xor, title="MLP (2 skryté neurony) — XOR vyřešeno")